In [1]:
import re
import copy
import pickle
import requests
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
final_df = pd.read_pickle("final_df.pickle")
final_df

,Ticker,Company,Fiscal_Date,Period_Ending,Transcript_Date,Transcript_Link,Announcement_Time,Key_People,Prepared_Remarks,QandA
0,NG,Novagold Resources Inc,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/novagold...,11:00 a.m.,[David Ottewell -- Vice President and Chief Fi...,"Thank you, Gillie, and good morning everyone. ...","[So, when you take into account proven and pro..."
1,SNX,SYNNEX Corporation,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/synnex-c...,5:00 p.m.,"[Marshall Witt -- Chief Financial Officer, Mar...","Thank you, Chantal, and good afternoon everyon...","[Hi Vince. Thanks for the question. Yeah, for ..."
2,PRGS,Progress Software,Q2 2020,"March 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/progress...,5:00 p.m.,"[Anthony Folger -- Chief Financial Officer, Yo...","Thank you, Nadia. Good afternoon, everyone, an...","[Thanks, Steve. I hope you're well, too., So S..."
3,NKE,Nike Inc,Q4 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/nike-inc...,5:00 p.m.,[John Donahoe -- President and Chief Executive...,"Thank you, operator. Hello, everyone, and than...","[Sure, Alexandra. In a funny way, I would char..."
4,CAMP,CalAmp Corp,Q1 2021,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/calamp-c...,4:30 p.m.,[Kurt Binder -- Executive Vice President and C...,Good afternoon and welcome to CalAmp's fiscal ...,"[Thanks, Mike., Yeah, Mike. So, as you pointed..."
...,...,...,...,...,...,...,...,...,...,...
967,GTYH,GTY Technology Holdings Inc.,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/gty-tech...,8:30 a.m.,[Tyler Radke -- Citigroup Inc. -- Citigroup In...,"Thank you, and good morning, everyone. I'm Joh...","[Hey, Scott, thanks for joining us today. So y..."
968,EOG,EOG Resources Inc,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/eog-reso...,10:00 a.m.,"[Billy Helms -- Chief Operating Officer, Ezra ...","Thanks, and good morning. Thanks for joining u...","[Yes. Leo, this is Bill. And I think we're goi..."
969,PRA,ProAssurance Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/proassur...,10:00 a.m.,[Edward Lewis Rand Jr. -- President and Chief ...,"Thank you, Jamie. In addition to our remarks o...",[I'm going to let Mike handle the bulk of that...
970,EXC,Exelon Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/exelon-c...,10:00 a.m.,[Joseph Nigro -- Senior Executive Vice Preside...,"Thank you, Mary. Good morning, everyone, and t...","[We have seen in other jurisdictions, legislat..."


In [37]:
final_df_corpuses = pd.DataFrame(final_df)
final_df_corpuses = final_df_corpuses.set_index('Ticker')
final_df_corpuses

,Company,Fiscal_Date,Period_Ending,Transcript_Date,Transcript_Link,Announcement_Time,Key_People,Prepared_Remarks,QandA
Ticker,,,,,,,,,
NG,Novagold Resources Inc,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/novagold...,11:00 a.m.,[David Ottewell -- Vice President and Chief Fi...,"Thank you, Gillie, and good morning everyone. ...","[So, when you take into account proven and pro..."
SNX,SYNNEX Corporation,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/synnex-c...,5:00 p.m.,"[Marshall Witt -- Chief Financial Officer, Mar...","Thank you, Chantal, and good afternoon everyon...","[Hi Vince. Thanks for the question. Yeah, for ..."
PRGS,Progress Software,Q2 2020,"March 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/progress...,5:00 p.m.,"[Anthony Folger -- Chief Financial Officer, Yo...","Thank you, Nadia. Good afternoon, everyone, an...","[Thanks, Steve. I hope you're well, too., So S..."
NKE,Nike Inc,Q4 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/nike-inc...,5:00 p.m.,[John Donahoe -- President and Chief Executive...,"Thank you, operator. Hello, everyone, and than...","[Sure, Alexandra. In a funny way, I would char..."
CAMP,CalAmp Corp,Q1 2021,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/calamp-c...,4:30 p.m.,[Kurt Binder -- Executive Vice President and C...,Good afternoon and welcome to CalAmp's fiscal ...,"[Thanks, Mike., Yeah, Mike. So, as you pointed..."
...,...,...,...,...,...,...,...,...,...
GTYH,GTY Technology Holdings Inc.,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/gty-tech...,8:30 a.m.,[Tyler Radke -- Citigroup Inc. -- Citigroup In...,"Thank you, and good morning, everyone. I'm Joh...","[Hey, Scott, thanks for joining us today. So y..."
EOG,EOG Resources Inc,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/eog-reso...,10:00 a.m.,"[Billy Helms -- Chief Operating Officer, Ezra ...","Thanks, and good morning. Thanks for joining u...","[Yes. Leo, this is Bill. And I think we're goi..."
PRA,ProAssurance Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/proassur...,10:00 a.m.,[Edward Lewis Rand Jr. -- President and Chief ...,"Thank you, Jamie. In addition to our remarks o...",[I'm going to let Mike handle the bulk of that...
